In [ ]:
#| default_exp tui


# TUI - Terminal User Interface

Textual-based terminal user interface components for audio transcription.


In [ ]:
#| export
from time import monotonic
from enum import Enum, auto
import asyncio

from textual import on, events
from textual.app import App, ComposeResult
from textual.reactive import reactive
from textual.containers import Container, VerticalGroup, Grid, HorizontalGroup, CenterMiddle, Horizontal
from textual.widgets import Header, Footer, Static, Digits, Button, Label, TextArea, Log, Select

from tui_writer.live import LiveTranscriber
from tui_writer.ai import has_session, start_session, apply_instruction, current_transcript, reset_session

In [ ]:
#| export

TEXTUAL_CSS = """
    Screen {
        background: $surface;
    }
    Header {
        background: $primary;
    }

    StatusCard {
        width: auto;
        row-span: 2;
        border: solid $primary;
        border-title-align: center;
        height: 100%;
        padding: 1 0;
    }

    #info-card {
        width: auto;
        border: solid #6b7280;
        border-title-align: center;
        height: 100%;
        column-span: 3;
        padding: 1;
    }

    #transcript-field {
        width: 100%;
        border: solid $warning;
        border-title-align: center;
        height: 100%;
        column-span: 3;
        padding: 1;
    }

    #top-cards {
        layout: grid;
        grid-size: 4 2;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
        margin: 1;
        height: 40vh;
    }

    #textfield {
        width: 100%;
        margin: 1;
        padding: 1;
        border: solid #8b5cf6;
        border-title-align: center;
        height: 50vh;
    }

    .info-text {
        margin-left: 1;
    }
    #whisper-select {
        margin-bottom: 1;
    }

    

    #status {
        content-align: center middle;
    }

    #status-time {
        border: double green;
        width: auto;
        padding: 0 1;
        margin: 1 0;
    }

    #media-controls {
        dock: bottom;
        align: center middle;
    }

    #media-controls Button {
        min-width: 20;
    }

    #pause,
    #resume,
    #stop {
        display: none;
    }

    .active #start {
        display: none;
    }
    .active #pause,
    .active #stop {
        display: block;
    }
    .active #status {
        color: $error;
    }

    .paused #start {
        display: none;
    }
    
    .paused #status {
        color: $warning;
    }

    .paused #resume,
    .paused #stop {
        display: block;
    }
"""

In [ ]:
#| export
class TimeDisplay(Digits): 
    """Custom time display widget""" 
    accumulated_time = 0 
    time_elapsed = reactive(0) 
    def on_mount(self): 
        self.update_timer = self.set_interval( 
            1 / 60, 
            self.update_time_elapsed, 
            pause=True 
        )
    
    def update_time_elapsed(self): 
        self.time_elapsed = self.accumulated_time + (
            monotonic() - self.start_time
        ) 
        
    def watch_time_elapsed(self): 
        time = self.time_elapsed 
        time, seconds = divmod(time, 60) 
        hours, minutes = divmod(time, 60) 
        time_string = f"{hours:02.0f}:{minutes:02.0f}:{seconds:05.2f}" 
        self.update(time_string) 
        
    def start(self): 
        self.start_time = monotonic() 
        self.update_timer.resume() 
        
    def pause(self): 
        self.accumulated_time = self.time_elapsed 
        self.update_timer.pause() 
        
    def stop(self): 
        self.start_time = monotonic() 
        self.accumulated_time = 0 
        self.time_elapsed = 0 
        self.update_timer.pause()

class StatusCard(Label):
    """Recording status card with controls"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Recording Status"

    def compose(self) -> ComposeResult:
        yield Static("○ STANDBY", id="status")
        with CenterMiddle():
            yield TimeDisplay("0:00:00", id="status-time")
        with HorizontalGroup(id="media-controls"):
            yield Button.error("● REC <space>", id="start")
            yield Button.success("⏸ PAUSE <space>", id="pause")
            yield Button.warning("▶ RESUME <space>", id="resume")
            yield Button("◼ STOP <s>", id="stop")

## TranscriptionTUI

Main terminal user interface application for audio transcription.

The TUI uses Textual's [theme system](https://textual.textualize.io/guide/design/) with built-in color variables like `$primary`, `$error`, `$success`, etc. 

**Available built-in themes:**
- `textual-dark` (default)
- `textual-light`
- `nord`
- `gruvbox`
- `tokyo-night`
- `solarized-light`

To change the theme, modify the `self.theme` assignment in `__init__`.


In [ ]:
#| export
class RecordingState(Enum):
    """High-level states the TUI can be in."""
    IDLE = auto()       # Not recording
    RECORDING = auto()  # Live transcription is running
    PAUSED = auto()     # Recording stopped but session kept
    EDIT = auto()       # Voice-based editing mode

class TranscriptionTUI(App):
    """
    Main Textual app for live transcription and AI-assisted editing.

    - Handles recording, pausing, resuming, and editing voice input in real time.
    - Integrates Whisper for transcription and an AI model for text editing.
    - Uses reactive state management (RecordingState) to update the UI automatically.

    The design principle: 
    - `_start()` and `_stop()` handle *recording logic only*.
    - `watch_state()` updates the UI reactively when state changes.
    - Event handlers (buttons, keybindings) change state and call those methods.
    """

    TITLE = "TUI Writer"
    SUB_TITLE = "Design Template"
    CSS = TEXTUAL_CSS
    AUTO_FOCUS = None

    # Available Whisper model options (label, value)
    WHISPER_MODELS = [
        ("Tiny      ⚡  Ultra fast   - Low accuracy", "tiny"),
        ("Base      ⚡  Fast         - Decent accuracy", "base"),
        ("Small     ⚖   Balanced    - Good accuracy", "small"),
        ("Medium    🐢  Slow        - High accuracy",  "medium"),
        ("Large     🐢  Very Slow   - Best accuracy", "large-v3"),
        ("Tiny.en   English-only    - faster", "tiny.en"),
        ("Base.en   English-only    - faster", "base.en"),
        ("Small.en  English-only    - faster", "small.en"),
        ("Medium.en English-only    - faster", "medium.en"),
        ("Large.en  English-only    - faster", "large-v3.en")
    ]
    WHISPER_LANGUAGES = [
        ("English", "en"),
        ("Norwegian", "no"),
        ("Swedish", "sv"),
        ("Danish", "da"),
        ("Finnish", "fi"),
        ("German", "de"),
        ("Dutch", "nl"),
        ("French", "fr"),
        ("Spanish", "es"),
        ("Portuguese", "pt"),
        ("Italian", "it"),
        ("Polish", "pl"),
        ("Czech", "cs"),
        ("Slovak", "sk"),
        ("Hungarian", "hu"),
        ("Greek", "el"),
        ("Turkish", "tr"),
        ("Russian", "ru"),
        ("Ukrainian", "uk"),
        ("Arabic", "ar"),
        ("Hebrew", "he"),
        ("Hindi", "hi"),
        ("Bengali", "bn"),
        ("Urdu", "ur"),
        ("Persian (Farsi)", "fa"),
        ("Thai", "th"),
        ("Vietnamese", "vi"),
        ("Indonesian", "id"),
        ("Malay", "ms"),
        ("Filipino (Tagalog)", "tl"),
        ("Chinese (Mandarin)", "zh"),
        ("Chinese (Cantonese)", "yue"),
        ("Japanese", "ja"),
        ("Korean", "ko"),
        ("Swahili", "sw"),
        ("Afrikaans", "af"),
        ("Romanian", "ro"),
        ("Bulgarian", "bg"),
        ("Serbian", "sr"),
        ("Croatian", "hr"),
        ("Slovenian", "sl"),
        ("Estonian", "et"),
        ("Latvian", "lv"),
        ("Lithuanian", "lt"),
    ]
    # Reactive state triggers watch_state() automatically when changed
    state = reactive(RecordingState.IDLE)

    # Keyboard bindings (TUI shortcuts)
    BINDINGS = [
        ("q", "quit", "Quit"),
        ("space", "toggle_recording", "Start/Pause/Resume"),
        ("s", "stop_recording", "Stop")
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Async background task for transcription loop
        self._task: asyncio.Task | None = None
        # The active LiveTranscriber instance
        self._transcriber: LiveTranscriber | None = None
        # Full transcript text accumulated so far
        self.all_chunks: str = ""
        # Default Whisper model name
        self.whisper_model = "base"
        # Default whisper language
        self.language = "en"

    def compose(self) -> ComposeResult:
        """Construct and layout all UI widgets."""
        # Saved to instance vars for easy reference in on_mount()
        self.info_card = Label(id="info-card")
        self.transcript_field = Label(id="transcript-field")
        self.textfield = Label(id="textfield")

        yield Header(show_clock=True)

        # Top section (status + info + current operations)
        with Container(id="top-cards"):
            yield StatusCard()
            with self.info_card:
                yield Static("Using Transcriber model:", classes="info-text")
                yield Select(
                    options=self.WHISPER_MODELS,
                    allow_blank=False,
                    value=self.whisper_model,
                    id="whisper-select",
                )
                yield Static("Language:", classes="info-text")
                yield Select(
                    options=self.WHISPER_LANGUAGES,
                    allow_blank=False,
                    value=self.language,
                    id="whisper-language"
                )
            yield self.transcript_field

        # Bottom section (main transcript log)
        with self.textfield:
            yield Log(id="main-textarea")
        yield Footer()

    # Action methods triggered by key bindings, refers to button-methods
    async def action_toggle_recording(self) -> None:
        """Spacebar: start if idle, pause if currently recording."""
        if self.state is not RecordingState.RECORDING:
            await self._start()
            self.state = RecordingState.RECORDING
        elif self.state is RecordingState.RECORDING:
            await self._stop()
            self.state = RecordingState.PAUSED
    
    async def action_stop_recording(self) -> None:
        """Key 's': fully stop and reset."""
        if self.state is not RecordingState.IDLE:
            await self._stop()
            self.state = RecordingState.IDLE

    # Button event handlers (mapped in the UI)
    @on(Button.Pressed, "#start")
    async def _on_start(self, _):
        """Start button pressed."""
        await self._start()
        self.state = RecordingState.RECORDING

    @on(Button.Pressed, "#pause")
    async def _on_pause(self, _):
        """Pause button pressed."""
        await self._stop()
        self.state = RecordingState.PAUSED

    @on(Button.Pressed, "#resume")
    async def _on_resume(self, _):
        """Resume button pressed."""
        await self._start()
        self.state = RecordingState.RECORDING

    @on(Button.Pressed, "#stop")
    async def _on_stop(self, _):
        """Stop button pressed."""
        await self._stop()
        self.state = RecordingState.IDLE

    # Whisper selectors
    @on(Select.Changed, "#whisper-select")
    def on_model_changed(self, event: Select.Changed) -> None:
        """Triggered when user changes Whisper model in dropdown."""
        self.whisper_model = str(event.value)

    @on(Select.Changed, "#whisper-language")
    def on_language_changed(self, event: Select.Changed) -> None:
        """Triggered when user changes Whisper language in dropdown."""
        self.language = str(event.value)
    
    # Keyboard shortcut for edit mode
    async def on_key(self, event: events.Key) -> None:
        """Press 'e' to toggle live edit mode."""
        if event.key != "e":
            return
        if self.state is not RecordingState.EDIT:
            # save current state before switching to edit mode
            self._old_state = self.state
            if self.state is RecordingState.RECORDING:
                await self._stop()
                self.state = RecordingState.PAUSED
            await self._start_edit() 
        else:
            # Leaving edit mode
            self.transcript_block.loading = True
            self.main_textarea.loading = True
            await self._stop_edit()
            self.transcript_block.loading = False
            self.main_textarea.loading = False
            self.plan = None
            self.edit_instructions = ""
            
            # Restore previous state safely
            self.state = getattr(self, "_old_state", RecordingState.PAUSED)
            # If previous state was recording, resume automatically.
            if self.state is RecordingState.RECORDING:
                await self._start()

    # Live edit mode helpers
    async def _start_edit(self) -> None:
        """Start edit session and initialize and start transcriber."""
        self.current_transcript = start_session(self.all_chunks)
        self._transcriber = LiveTranscriber(
            model_id=self.whisper_model,
            language=self.language,
            on_transcript=self.on_edit_transcript,
            vad_threshold=0.5,
            min_speech_duration_ms=250,
            min_silence_duration_ms=500,
        )
        self._task = asyncio.create_task(self._transcriber.start())
        self.state = RecordingState.EDIT

    async def _stop_edit(self) -> None:
        """Stop edit mode and apply queued voice edit instructions."""
        try:
            if self._transcriber:
                self._transcriber.stop()
            if self._task:
                await self._task
            if self._transcriber:
                await self._transcriber.flush()
        finally:
            self._transcriber = None
            self._task = None
            # Apply accumulated voice instructions to current transcript
            self.current_transcript, self.plan = apply_instruction(self.edit_instructions)
            reset_session()
            self.all_chunks = self.current_transcript
            # Show resulting plan and updated transcript in UI
            ops_text = f"Plan: {self.plan.model_dump_json(indent=2)}"
            self.transcript_block.update(ops_text)
            self.main_textarea.clear()
            self.main_textarea.write_lines(self.all_chunks.splitlines(True))

    # Transcription lifecycle
    async def _start(self) -> None:
        """Initialize and start live transcription."""
        self._transcriber = LiveTranscriber(
            model_id=self.whisper_model, language=self.language,
            on_transcript=self.on_transcript_chunk,
            vad_threshold=0.5, min_speech_duration_ms=250, min_silence_duration_ms=500,
        )
        self._task = asyncio.create_task(self._transcriber.start())

    async def _stop(self) -> None:
        """Stop active transcriber and cleanup task."""
        try:
            if self._transcriber:
                self._transcriber.stop()
            if self._task:
                await self._task
            if self._transcriber:
                await self._transcriber.flush()
        finally:
            self._transcriber = None
            self._task = None

    # Callback handlers
    def on_transcript_chunk(self, text: str) -> None:
        """Called whenever the transcriber produces a new text chunk."""
        text = (text or "").strip()
        if not text or self.state is RecordingState.IDLE:
            return
        self.all_chunks += text + "\n"
        self.main_textarea.write_line(text)
    
    def on_edit_transcript(self, instruction: str) -> None:
        """Collects spoken edit instructions during edit mode."""
        instruction = (instruction or "").strip()
        if not instruction:
            return
        prev = getattr(self, "edit_instructions", "")
        self.edit_instructions = (prev + ("\n" if prev else "") + instruction)

    # UI state watcher (reactive)
    def watch_state(self, new_state: RecordingState) -> None:
        """Reactively update UI when recording state changes."""
        self.status_card.remove_class("active")
        self.status_card.remove_class("paused")

        match new_state:
            case RecordingState.IDLE:
                self.status_text.update("○ STANDBY")
                self.time_display.stop()
            case RecordingState.RECORDING:
                self.status_card.add_class("active")
                self.status_text.update("● RECORDING...")
                self.time_display.start()
            case RecordingState.PAUSED:
                self.status_card.add_class("paused")
                self.status_text.update("⏸ PAUSED")
                self.time_display.pause()
            case RecordingState.EDIT:
                self.status_card.add_class("active")
                self.status_text.update("● LIVE EDITING...")
                self.time_display.pause()

    # Mount / Unmount lifecycle
    def on_mount(self) -> None:
        """Initialize widget references and set titles."""
        self.status_card: StatusCard = self.query_one(StatusCard)
        self.status_text: Static = self.query_one("#status", Static)
        self.time_display: TimeDisplay = self.query_one(TimeDisplay)
        self.transcript_block: Label = self.query_one("#transcript-field", Label)
        self.main_textarea: Log = self.query_one("#main-textarea", Log)
        self.transcript_field.border_title = "Edit Operations"
        self.info_card.border_title = "Info & Settings"
        self.textfield.border_title = "Full Transcript"

    async def on_unmount(self) -> None:
        """Cleanly stop background tasks when app closes."""
        await self._stop()